In [1]:
!pip install datasets transformers tensorflow numpy matplotlib opencv-python scikit-learn


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 35.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 20.2 MB/s eta 0:00:00


In [2]:
import numpy as np
from PIL import Image
from datasets import load_dataset

# تحميل مجموعة البيانات
dataset = load_dataset("yasimed/lung_cancer")

# دالة استخراج التصنيفات من النصوص
def extract_label(text):
    return 1 if "cancer" in text.lower() else 0  # تحويل النص إلى تصنيف رقمي

# استخراج الصور والتصنيفات مع التأكد من الاتساق
images = []
labels = []

for item in dataset['train']:
    # تحويل الصورة إلى كائن PIL
    image = item['image'].convert("RGB")  # تحويل الصورة إلى 3 قنوات (RGB)

    # تغيير الحجم إلى 224x224
    image = image.resize((224, 224))

    # تحويل الصورة إلى مصفوفة NumPy وتطبيع القيم بين 0 و 1
    image = np.array(image, dtype=np.float32) / 255.0

    images.append(image)
    labels.append(extract_label(item['text']))  # استخراج التصنيف من النص

# تحويل القوائم إلى مصفوفات NumPy بعد التأكد من التجانس
images = np.array(images, dtype=np.float32)
labels = np.array(labels, dtype=np.int32)

# التحقق من الأشكال
print("✅ شكل الصور:", images.shape)  # يجب أن يكون (613, 224, 224, 3)
print("✅ شكل التصنيفات:", labels.shape)  # يجب أن يكون (613,)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/313 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/59.4M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/613 [00:00<?, ? examples/s]

✅ شكل الصور: (613, 224, 224, 3)
✅ شكل التصنيفات: (613,)


In [3]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np


In [4]:
from sklearn.model_selection import train_test_split

# تقسيم البيانات إلى تدريب واختبار
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# طباعة حجم البيانات بعد التقسيم
print("✅ عدد صور التدريب:", X_train.shape[0])
print("✅ عدد صور الاختبار:", X_test.shape[0])


✅ عدد صور التدريب: 490
✅ عدد صور الاختبار: 123


In [5]:
# إنشاء محول بيانات التدريب
train_datagen = ImageDataGenerator(
    rotation_range=20,  # تدوير عشوائي حتى 20 درجة
    width_shift_range=0.1,  # إزاحة أفقية بنسبة 10%
    height_shift_range=0.1,  # إزاحة رأسية بنسبة 10%
    horizontal_flip=True,  # قلب الصورة أفقيًا
)

# تجهيز البيانات للتدريب والاختبار
train_generator = train_datagen.flow(X_train, y_train, batch_size=32, shuffle=True)
test_generator = ImageDataGenerator().flow(X_test, y_test, batch_size=32, shuffle=False)


In [9]:
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Activation, Add
from tensorflow.keras.layers import MaxPooling2D, GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.layers import SeparableConv2D, SpatialDropout2D, concatenate
from tensorflow.keras.layers import Multiply, Reshape, GlobalMaxPooling2D
from tensorflow.keras.regularizers import l2
from tensorflow.keras.models import Model
import tensorflow as tf

def enhanced_residual_block(x, filters, expansion=4, stochastic_depth_rate=0.1):
    shortcut = x
    in_channels = x.shape[-1]

    # Bottleneck expansion
    x = Conv2D(filters, 1, padding='same', use_bias=False)(x)
    x = BatchNormalization()(x)
    x = Activation('swish')(x)

    # Depthwise Convolution
    x = SeparableConv2D(filters, 3, padding='same', use_bias=False)(x)
    x = BatchNormalization()(x)
    x = Activation('swish')(x)

    # Squeeze-and-Excitation
    se = GlobalAveragePooling2D()(x)
    se = Dense(filters//4, activation='swish')(se)
    se = Dense(filters, activation='sigmoid')(se)
    se = Reshape((1, 1, filters))(se)
    x = Multiply()([x, se])

    # Projection
    x = Conv2D(filters*expansion, 1, padding='same', use_bias=False)(x)
    x = BatchNormalization()(x)

    if tf.keras.backend.random_uniform([]) < stochastic_depth_rate:
        return shortcut

    if in_channels != filters*expansion:
        shortcut = Conv2D(filters*expansion, 1, padding='same')(shortcut)
        shortcut = BatchNormalization()(shortcut)

    x = Add()([x, shortcut])
    x = Activation('swish')(x)
    return x

def build_hybrid_model(input_shape=(224, 224, 3), num_classes=1):  # تم التعديل هنا
    inputs = Input(shape=input_shape)

    # Stem Layer
    x = Conv2D(64, (7, 7), strides=2, padding='same',
              kernel_regularizer=l2(1e-4))(inputs)
    x = BatchNormalization()(x)
    x = Activation('swish')(x)
    x = MaxPooling2D(3, strides=2, padding='same')(x)

    # Hybrid Backbone
    filters = [64, 128, 256, 512]  # تم تخفيض المراحل لتتناسب مع 224x224
    repetitions = [3, 4, 6, 3]     # إجمالي 16 كتلة

    for i, (f, r) in enumerate(zip(filters, repetitions)):
        if i > 0:
            x = Conv2D(f, 3, strides=2, padding='same')(x)
            x = BatchNormalization()(x)
            x = SpatialDropout2D(0.1*(i+1))(x)

        for _ in range(r):
            x = enhanced_residual_block(x, f, expansion=2)

    # Multi-Scale Feature Fusion
    x1 = GlobalAveragePooling2D()(x)
    x2 = GlobalMaxPooling2D()(x)
    x = concatenate([x1, x2])

    # Classifier Head
    x = Dense(1024, activation='swish', kernel_regularizer=l2(1e-4))(x)
    x = Dropout(0.5)(x)
    x = Dense(512, activation='swish', kernel_regularizer=l2(1e-4))(x)
    x = Dropout(0.3)(x)
    outputs = Dense(num_classes, activation='sigmoid')(x)

    model = Model(inputs, outputs)

    optimizer = tf.keras.optimizers.AdamW(
        learning_rate=1e-4,
        weight_decay=1e-5
    )

    model.compile(
        optimizer=optimizer,
        loss='binary_crossentropy',
        metrics=['accuracy', tf.keras.metrics.AUC(name='auc')]
    )

    return model

# إنشاء النموذج
model = build_hybrid_model()
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2             │ (None, 224, 224, 3)    │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_52 (Conv2D)        │ (None, 112, 112, 64)   │          9,472 │ input_layer_2[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_73    │ (None, 112, 112, 64)   │            256 │ conv2d_52[0][0]        │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_62             │ (None, 112, 112, 64)   │              0 │ batch_normalization_7… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_2           │ (None, 56, 56, 64)     │              0 │ activation_62[0][0]    │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_53 (Conv2D)        │ (None, 56, 56, 64)     │          4,096 │ max_pooling2d_2[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_74    │ (None, 56, 56, 64)     │            256 │ conv2d_53[0][0]        │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_63             │ (None, 56, 56, 64)     │              0 │ batch_normalization_7… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ separable_conv2d_21       │ (None, 56, 56, 64)     │          4,672 │ activation_63[0][0]    │
│ (SeparableConv2D)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_75    │ (None, 56, 56, 64)     │            256 │ separable_conv2d_21[0… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_64             │ (None, 56, 56, 64)     │              0 │ batch_normalization_7… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ global_average_pooling2d… │ (None, 64)             │              0 │ activation_64[0][0]    │
│ (GlobalAveragePooling2D)  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_45 (Dense)          │ (None, 16)             │          1,040 │ global_average_poolin… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_46 (Dense)          │ (None, 64)             │          1,088 │ dense_45[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ reshape_20 (Reshape) 

 Total params: 11,652,017 (44.45 MB)

 Trainable params: 11,620,145 (44.33 MB)

 Non-trainable params: 31,872 (124.50 KB)

In [10]:
# تدريب النموذج
history = model.fit(train_generator, validation_data=test_generator, epochs=40)

# حفظ النموذج بعد التدريب
model.save("CKKT.h5")


Epoch 1/40
16/16 ━━━━━━━━━━━━━━━━━━━━ 121s 3s/step - accuracy: 0.7579 - auc: 0.7566 - loss: 1.4037 - val_accuracy: 0.7805 - val_auc: 0.5000 - val_loss: 0.8582
Epoch 2/40
16/16 ━━━━━━━━━━━━━━━━━━━━ 6s 354ms/step - accuracy: 0.8789 - auc: 0.8861 - loss: 0.7837 - val_accuracy: 0.7805 - val_auc: 0.5000 - val_loss: 0.8102
Epoch 3/40
16/16 ━━━━━━━━━━━━━━━━━━━━ 7s 401ms/step - accuracy: 0.9020 - auc: 0.9402 - loss: 0.5929 - val_accuracy: 0.7805 - val_auc: 0.5000 - val_loss: 0.7758
Epoch 4/40
16/16 ━━━━━━━━━━━━━━━━━━━━ 10s 398ms/step - accuracy: 0.9239 - auc: 0.9385 - loss: 0.7069 - val_accuracy: 0.7805 - val_auc: 0.5000 - val_loss: 0.7517
Epoch 5/40
16/16 ━━━━━━━━━━━━━━━━━━━━ 6s 350ms/step - accuracy: 0.8818 - auc: 0.9468 - loss: 0.8627 - val_accuracy: 0.7805 - val_auc: 0.5000 - val_loss: 0.7376
Epoch 6/40
16/16 ━━━━━━━━━━━━━━━━━━━━ 6s 400ms/step - accuracy: 0.9332 - auc: 0.9314 - loss: 0.5540 - val_accuracy: 0.7805 - val_auc: 0.5000 - val_loss: 0.7311
Epoch 7/40
16/16 ━━━━━━━━━━━━━━━━━━━━ 6s

In [12]:
# التعديل الصحيح لاستقبال جميع القيم
results = model.evaluate(test_generator)
print(f"Loss: {results[0]:.4f}, Accuracy: {results[1]:.4f}, AUC: {results[2]:.4f}")

# أو باستخدام أسماء المتغيرات المحددة
test_metrics = model.evaluate(test_generator)
loss = test_metrics[0]
accuracy = test_metrics[1]
auc_score = test_metrics[2]

print(f"✅ دقة النموذج: {accuracy*100:.2f}% | AUC: {auc_score:.4f}")

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step - accuracy: 0.7882 - auc: 0.5186 - loss: 1.6605
Loss: 1.7113, Accuracy: 0.7805, AUC: 0.5185
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - accuracy: 0.7882 - auc: 0.5186 - loss: 1.6605
✅ دقة النموذج: 78.05% | AUC: 0.5185


In [ ]:
import numpy as np
from PIL import Image

def preprocess_image(image_path):
    """ تحميل الصورة وتحويلها إلى الشكل المناسب للنموذج """
    image = Image.open(image_path).convert("RGB")  # تحويل إلى 3 قنوات (RGB)
    image = image.resize((224, 224))  # تغيير الحجم إلى 224x224
    image = np.array(image, dtype=np.float32) / 255.0  # تطبيع القيم بين 0 و 1
    image = np.expand_dims(image, axis=0)  # إضافة بعد جديد ليتناسب مع الإدخال المتوقع (batch_size, 224, 224, 3)
    return image

# تجربة الصورة
image_path = "/content/image.png"  # ضع مسار الصورة هنا
processed_image = preprocess_image(image_path)

# التنبؤ باستخدام النموذج
prediction = model.predict(processed_image)

# تفسير النتيجة بناءً على النص الموجود في البيانات
def interpret_prediction(pred):
    """ تحويل التنبؤ الرقمي إلى نص بشري """
    if pred >= 0.5:
        return "مصاب بسرطان الرئة 🩸"
    else:
        return "غير مصاب بالسرطان ✅"

# عرض النتيجة النصية مع احتمالية الإصابة
result = interpret_prediction(prediction[0][0])
print(f"🔍 النتيجة: {result} (احتمال الإصابة: {prediction[0][0] * 100:.2f}%)")
